In [168]:
!pip install -q transformers

In [169]:

!pip install biopython

In [170]:

#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

In [171]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple

from Bio import Entrez
from Bio.PDB import PDBList
import pandas as pd

from transformers import BertModel, BertTokenizer
import re

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split

import pickle

Retrieving data from pdb bank using pisces ids with filter

In [172]:


def download_pdb_files(pdb_ids, output_dir):
  pdbl = PDBList()
  for pdb_id in pdb_ids:
    pdb_id_trimmed = pdb_id[:-1] #the last letter is not pdb id, its the chain we need to take
    pdbl.retrieve_pdb_file(pdb_id_trimmed, pdir=output_dir, file_format='mmCif') #maybe change it to mm.cif?

file_path = '/content/cullpdb_pc30.0_res0.0-2.0_noBrks_noDsdr_len40-10000_R0.25_Nmr_d2024_06_13_chains2636'
df = pd.read_csv(file_path, delim_whitespace=True)

#pdb_ids = df['PDBchain'].tolist()

df = df[df['len'] <= 128]
pdb_ids = df['PDBchain'].tolist()
print(pdb_ids)

download_pdb_files(pdb_ids, '/content/target directory')

['1E0ZA', '1RY4A', '1UG7A', '1WHNA', '2AYAA', '2D87A', '2DB5A', '2DKPA', '2EBKA', '2ETTA', '2FM4A', '2I4KA', '2JQ5A', '2K0RA', '2K5TA', '2LR4A', '2LS01', '2LVLA', '2MAHA', '2NCNA', '2RLOA', '2RV8A', '2YTUA', '4AR0A', '5AIWA', '5H7UA', '5N9QA', '8E22X', '1CMOA', '1DDFA', '1EALA', '1PQNA', '1QKLA', '1UEYA', '1W2QA', '1WH4A', '1WJRA', '1X5MA', '1ZU1A', '2CSOA', '2DMKA', '2EE7A', '2L1LB', '2LOEA', '2MF7A', '2MPLA', '2NBOA', '2TMPA', '4UEIA', '5M1UA', '7XRWA', '8OILA', '1RSFA', '1V5SA', '1X5EA', '1ZO0A', '2A4HA', '2CR4A', '2DAVA', '2IF1A', '2JP2A', '2JVNA', '2K2VA', '2L3GA', '2L57A', '2LBCA', '2LLNA', '2M26A', '3MSPA', '7F7NA', '7KNVA', '8SQXA', '1BUQA', '1BW3A', '1FR0A', '1GIOA', '1MH6A', '1PSYA', '1UENA', '1WYJA', '2AI6A', '2DHJA', '2K4VA', '2K4ZA', '2KA5A', '2KFPA', '2KYTA', '2L2CA', '2LDUA', '2LM0A', '2MDKA', '2MV4A', '2N2HB', '2NBGA', '2PRFA', '4A53A', '5KNWA', '5T3YA', '5WOEA', '6GWMA', '6TH8A', '6UF2A', '6XEHA', '7BQCA', '7PRDA', '7QYIA', '1DC7A', '1E8EA', '1EO1A', '1PIRA', '1T3VA', 

KeyboardInterrupt: 

creating a dictionary of continuous chains

In [ ]:
#we need the chain acces information
pdb_id_map = {pdb_id[:-1]: pdb_id for pdb_id in pdb_ids}
print(pdb_id_map)

mapping_file = os.path.join('/content/map', 'pdb_id_mapping.csv')
#pd.Series(pdb_id_map).to_csv(mapping_file, header=False)
pd.Series({k.lower(): v for k, v in pdb_id_map.items()}).to_csv(mapping_file, header=False) # un capitalising them

Encoding protein sequences and extracting angles and embeddings into a file for later use

In [191]:
from Bio.PDB import PDBParser
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def is_chain_continuous(chain):
    prev_resnum = None
    for residue in chain:
        if prev_resnum is not None and residue.get_id()[1] != prev_resnum + 1:
            return False
        prev_resnum = residue.get_id()[1]
    return True

def extract_input(pdb_file, model, tokenizer):
    #parser = PDBParser()
    #structure = parser.get_structure("protein", pdb_file)
    #model0 = structure[0]

    file_model = file_path.replace(".cif","")
    pdbl = PDBList()
    cif_parser = MMCIFParser()
    structure = cif_parser.get_structure("protein", pdb_file)
    model0 = structure[0]


    mapping_file = os.path.join('/content/map', 'pdb_id_mapping.csv')
    pdb_id_map = pd.read_csv(mapping_file, header=None, index_col=0).squeeze().to_dict()

    #extracting the file name as it contains the id
    filename = os.path.basename(pdb_file)
    pdb_id = filename[:4]
    pdb_id = pdb_id
    #print(pdb_id)

    full_pdb_id = pdb_id_map.get(pdb_id) #pdb id with the correct chain


    chain_A = model0[full_pdb_id[-1]]  # and we get chain A

    # dictionary converting 3-letter codes to 1-letter codes

    #try:
    #    chain_A = model0['A']
    #except KeyError:
    #    print(f"Chain A not found in {pdb_file}")
    #    return None, None, None, None, None

    #if not is_chain_continuous(chain_A):
    #    print(f"Chain A in {pdb_file} is discontinuous")
    #    return None, None, None, None, None

    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    #print(''.join(sequence))
    seq_example =  ' '.join(sequence)
    #return seq_example

#def extract_coordinates(pdb_file):
    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

    chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

    cnt = 1
    phi_angles = {}
    phi_angles_list = []
    psi_angles = {}
    psi_angles_list = []

    for key in d:
      if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
          phi_angles[key] = d[key].angle
          phi_angles_list.append(d[key].angle)
      elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
          psi_angles[key] = d[key].angle
          psi_angles_list.append(d[key].angle)


    structure.internal_to_atom_coordinates(verbose = False)#before it was true, you can change this back if theres any
    io = PDBIO() #this is to write a pdb file again
    io.set_structure(structure)#set structure, the structure you wan tin the pdb file
    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = np.vstack((psi,phi))
    angle_tensor = torch.tensor(angles, dtype=torch.float32)

    #the ones im commenting out must be uncommented for our custom transformer
    #tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )

    seq = re.sub(r"[UZOB]", "X", seq_example)

    encoded_input = tokenizer(seq, return_tensors='pt')
    #output = bert_model(**encoded_input)
    #embedded_pb = output.last_hidden_state
    #N, D = embedded_pb.size()[1], embedded_pb.size()[2]
    #embeddings = embedded_pb

    #from here on im directly using libraries to implement transformers
    #model_name = "facebook/esm2_t33_650M_UR50D"
    #model = AutoModelForSequenceClassification.from_pretrained(model_name)
    #tokenizer = AutoTokenizer.from_pretrained(model_name)

    #inputs = tokenizer.encode(
    #    seq,
    #    add_special_tokens=True,
    #    max_length=1024,
    #    return_attention_mask=True,
    #    return_tensors='pt',
    #    truncation = True #not sure if this is required
    #)

    #Make predictions
    outputs = model(**encoded_input)

    embeddings = outputs.last_hidden_state

    #print('N:', N,D)
    return seq, angle_tensor, encoded_input, outputs, embeddings

In [ ]:
#model_name = "facebook/esm2_t33_650M_UR50D" #here we can change to our mmodel and our tokenizer later
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model_embedd = BertModel.from_pretrained("Rostlab/prot_bert")

# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 20
#model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
#model.forward(embeddings)
#criterion = MeanAngleErrorLoss()
#num_epochs = 20
#criterion = nn.MSELoss()
#criterion = nn.L1Loss()

#criterion  = AngularLoss()
#trainer = TransformerTrainer4(model, criterion, num_epochs, embedded_pb, angles)
#trainer.train()






In [ ]:
len(df)

Extracting input files

If i try to run this code for alll the files in one go, it crashes, so run this multiple times and it will continue from where it stopped.

In [188]:
import os
import torch

directory_path = '/content/target directory'
target_path = '/content/map'
embeddings_file = os.path.join(target_path, 'embeddings.pt')
angles_file = os.path.join(target_path, 'angles.pt')
sequences_file = os.path.join(target_path, 'sequences.pkl')

# Load existing tensors if they exist
if os.path.exists(embeddings_file) and os.path.exists(angles_file):
    embeddings = torch.load(embeddings_file)
    angles = torch.load(angles_file)
    with open(sequences_file, 'rb') as f:
        sequences = pickle.load(f)
    start_index = embeddings.size(0)

else:
    #notice i have padded embeddings so they are all the same size. i am not sure if that will cause issues
    embeddings = torch.zeros(0, 130, 1024) # embeddings max size is 130, because you have astart and an end token

    angles = torch.zeros(0, 2, 130)
    sequences = []
    start_index = 0

# Number of files to process in this run
num_files_to_process = 2

# Iterate over the files to process

i = start_index

def pad_sequence(sequence, target_length=128, pad_char="X"): # may not need this as im using a padding in the tokenizer itself
    return sequence + (pad_char * (target_length - len(sequence)))


for filename in os.listdir(directory_path):
    if i >= start_index + num_files_to_process:
        break

    file_path = os.path.join(directory_path, filename)

    if os.path.isfile(file_path):
        sequence, angle, inputs, outputs, embedding = extract_input(file_path, model_embedd, tokenizer)
        if sequence is None or embedding is None or angle is None:
            print(f"Skipping {file_path} due to missing data")
            continue
        padded_sequence = pad_sequence(sequence)  # may not need this as im using a padding in the tokenizer itself
        sequences.append(padded_sequence)

        # Append the new data to the existing tensors
        embedding_length = min(embedding.size(1), 130)
        if i >= embeddings.size(0):
            embeddings = torch.cat([embeddings, torch.zeros(1, 130, 1024)], dim=0)
        embeddings[i, :embedding_length] = embedding[0, :, :]

        angle_length = min(angle.size(1), 130)
        if i >= angles.size(0):
            angles = torch.cat([angles, torch.zeros(1, 2, 130)], dim=0)
        angles[i, :, :angle_length] = angle[:, :]

        i += 1
        print(i)

# Save the updated tensors back to the files
torch.save(embeddings, embeddings_file)
torch.save(angles, angles_file)
with open(sequences_file, 'wb') as f:
    pickle.dump(sequences, f)


17
18


In [180]:
angles = torch.load('/content/map/angles.pt')
phi_angles = angles[:,0,:]
psi_angles = angles[:,1,:]

print(angles.size())
print(phi_angles.size())
print(angles[0,:,:])

torch.Size([16, 2, 130])
torch.Size([16, 130])
tensor([[-179.9597, -179.9776, -179.9898, -179.5931, -179.9696,  179.9690,
          179.7876, -179.5145,  179.9680, -179.7181,  178.7509, -179.6375,
         -179.6294,  178.6904, -179.0989,  179.6593, -178.3506, -179.0738,
         -179.9825, -178.6783,  178.5564,  179.1216,  178.6022, -178.2576,
         -179.2678,  178.3880,  178.4346,  179.4729,  179.0067,  179.5735,
          178.8113, -179.8473, -179.8472,  178.9480, -179.6333, -179.2733,
         -178.7039, -179.5098,  179.7791,  179.8909,  179.7663,  179.8944,
         -178.8563, -178.9677,  179.2875, -179.3298,  179.3405, -179.9337,
         -179.8166, -179.4043, -178.3544, -178.2878, -178.0784, -179.3499,
         -178.9776, -179.4838, -179.3712, -179.8165,  179.8505, -179.5029,
          179.5874, -179.3981, -179.1741, -179.7368, -179.9230, -179.9583,
         -179.9998, -179.9452,  179.9397, -179.8857, -179.8627,  179.7673,
          179.9326,  179.7460, -179.8444,  179.4500, 

In [181]:
embeddings = torch.load('/content/map/embeddings.pt')
print(embeddings.size())

torch.Size([16, 130, 1024])


In [182]:
target_path = '/content/map'
sequences_file = os.path.join(target_path, 'sequences.pkl')  # Ensure this matches the file path used when saving
with open(sequences_file, 'rb') as f:
        sequences = pickle.load(f)
print(sequences)
print(len(sequences))

['G A M D P E F N K A I F T V D A K T T E I L V A N D K A C G L L G Y S S Q D L I G Q K L T Q F F L R S D S D V V E A L S E E H M E A D G H A A V V F G T V V D I I S R S G E K I P V S V W M K R M R Q E R R L C C V V V L E P V E R', 'G S S G S S G A G Q V V H T E T T E V V L T A D P V T G F G I Q L Q G S V F A T E T L S S P P L I S Y I E A D S P A E R C G V L Q I G D R V M A I N G I P T E D S T F E E A N Q L L R D S S I T S K V T L E I E F D V A E S V I P S S G S G P S S G', 'G A M D P E F N K A I F T V D A K T T E I L V A N D K A C G L L G Y S S Q D L I G Q K L T Q F F L R S D S D V V E A L S E E H M E A D G H A A V V F G T V V D I I S R S G E K I P V S V W M K R M R Q E R R L C C V V V L E P V E R', 'G S S G S S G A G Q V V H T E T T E V V L T A D P V T G F G I Q L Q G S V F A T E T L S S P P L I S Y I E A D S P A E R C G V L Q I G D R V M A I N G I P T E D S T F E E A N Q L L R D S S I T S K V T L E I E F D V A E S V I P S S G S G P S S G', 'S M Q P E E F A A A A R G G F G G D A E K 

Dividing data into test and validation sets


In [183]:
total_size = len(angles)
train_size = int(total_size) #80 percent of data
val_size = int(0.1*total_size)
test_size = total_size - train_size - val_size

dataset = TensorDataset(angles,embeddings)

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])


With masking, experimental, I dont know if this works properly, it takes longer to train, dont run this now

In [ ]:

class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor: # added mask criteria
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf')) #if masked, the value is going as low as possible to avoid being scored

        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output


class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor: # masking criteria
        attention_output = self.self_attention(embeddings, mask) #mask included
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output

def pad_angles(angles, target_length=128): #wherever we pad the angles we are masking it
    padded_angles = torch.zeros(angles.size(0), target_length)
    angle_length = min(angles.size(1), target_length)
    padded_angles[:, :angle_length] = angles[:, :angle_length]
    return padded_angles

def create_mask(lengths, max_len): #wherever we pad the sequence we are masking it
    mask = torch.arange(max_len).expand(len(lengths), max_len) < lengths.unsqueeze(1)
    return mask

class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor, mask): # added mask here
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)*mask # masking
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi)) / mask.sum() #masking

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)*mask # masking
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi)) /mask.sum() # masking

        loss = loss_phi + loss_psi
        print(loss)
        return loss


class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor, mask: torch.Tensor): # mask
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.mask = mask # mask
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence, self.mask)[:, :len(self.angles_tensor)] # mask i didnt include one mrore feature, if there is an error include this [:, :self.angles_tensor.size(1)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor, self.mask.squeeze()) # mask
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss

Without masking, our usual training

In [192]:

class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output


class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output

class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor):
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

        loss = loss_phi + loss_psi
        print(loss)
        return loss

In [194]:

class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss

In [196]:
source_dir = '/content/map'

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 100
dropout_rate = 0.1
model = TransformerModel(embed_dim=1024, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate)
criterion = AngularLoss()

N= 130
D = 1024

for i in range(len(sequences)): # we can replace this with something better once we get all the files
  angles_tensor = (angles[i,:,:].T)*(np.pi/180)
  mask = create_mask(torch.tensor([130]), 130)
  trainer = TransformerTrainer(model, criterion, num_epochs, embeddings[i,:,:] , angles_tensor, mask)
  print(i)
  trainer.train()


0
tensor(0.0307, grad_fn=<AddBackward0>)
tensor(0.0210, grad_fn=<AddBackward0>)
tensor(0.0212, grad_fn=<AddBackward0>)
tensor(0.0160, grad_fn=<AddBackward0>)
tensor(0.0196, grad_fn=<AddBackward0>)
tensor(0.0201, grad_fn=<AddBackward0>)
tensor(0.0174, grad_fn=<AddBackward0>)
tensor(0.0136, grad_fn=<AddBackward0>)
tensor(0.0166, grad_fn=<AddBackward0>)
tensor(0.0141, grad_fn=<AddBackward0>)
tensor(0.0146, grad_fn=<AddBackward0>)
tensor(0.0151, grad_fn=<AddBackward0>)
tensor(0.0136, grad_fn=<AddBackward0>)
tensor(0.0141, grad_fn=<AddBackward0>)
tensor(0.0143, grad_fn=<AddBackward0>)
tensor(0.0130, grad_fn=<AddBackward0>)
tensor(0.0135, grad_fn=<AddBackward0>)
tensor(0.0139, grad_fn=<AddBackward0>)
tensor(0.0127, grad_fn=<AddBackward0>)
tensor(0.0130, grad_fn=<AddBackward0>)
tensor(0.0129, grad_fn=<AddBackward0>)
tensor(0.0120, grad_fn=<AddBackward0>)
tensor(0.0119, grad_fn=<AddBackward0>)
tensor(0.0122, grad_fn=<AddBackward0>)
tensor(0.0114, grad_fn=<AddBackward0>)
tensor(0.0117, grad_fn=

KeyboardInterrupt: 

In [189]:

angles_tensor = angles[2,:,:].T*(np.pi/180)
predicted_angles = model.forward(embeddings[2,:,:])[:, :len(angles_tensor)]

predicted_angles

tensor([[-3.3094, -0.1151],
        [-3.2445, -0.5209],
        [-3.2646, -0.2964],
        [-3.1630, -0.1722],
        [-3.1842, -0.2536],
        [-3.1599, -0.1845],
        [-3.1393, -0.3063],
        [-3.1800, -0.2026],
        [-3.2558,  2.2137],
        [-3.1455,  2.2974],
        [-3.1891,  1.9418],
        [-3.2515,  2.2833],
        [-3.1471,  2.2670],
        [-3.2474,  2.5099],
        [-3.2548,  2.1943],
        [-3.0779, -0.4601],
        [-3.1055, -1.1975],
        [-3.0820, -0.2375],
        [-3.2312, -0.6816],
        [-3.0857,  2.1918],
        [-3.1991,  2.1065],
        [-3.2185,  2.1351],
        [-3.1309,  2.2651],
        [-3.2589,  2.2272],
        [-3.1510,  2.1484],
        [-3.1898, -0.2442],
        [-3.2161, -0.4022],
        [-3.2605, -0.2601],
        [-3.1902, -1.2207],
        [-3.2336,  0.1664],
        [-3.1310, -0.0980],
        [-3.1253, -0.0787],
        [-3.2037,  0.6948],
        [-3.1618, -0.9327],
        [-3.2205,  2.4689],
        [-3.1034, -0

Notice the padding

In [190]:
angles_tensor

tensor([[-3.1409,  0.7185],
        [-3.1412,  0.4480],
        [-3.1414, -1.8298],
        [-3.1345, -0.8967],
        [-3.1411, -0.4248],
        [ 3.1411, -0.2205],
        [ 3.1379, -0.9062],
        [-3.1331,  0.7970],
        [ 3.1410,  1.9017],
        [-3.1367,  1.5322],
        [ 3.1198,  2.0018],
        [-3.1353,  1.9091],
        [-3.1351,  2.1358],
        [ 3.1187,  3.0654],
        [-3.1259,  2.6813],
        [ 3.1356, -0.4145],
        [-3.1128, -1.1787],
        [-3.1254, -0.0106],
        [-3.1413, -1.9177],
        [-3.1185,  1.6688],
        [ 3.1164,  1.9533],
        [ 3.1263, -1.1191],
        [ 3.1172,  2.8920],
        [-3.1112,  2.6147],
        [-3.1288,  3.0419],
        [ 3.1135, -0.0943],
        [ 3.1143, -1.1155],
        [ 3.1324, -0.8819],
        [ 3.1243, -0.7248],
        [ 3.1341, -0.6675],
        [ 3.1208, -0.9783],
        [-3.1389, -1.2810],
        [-3.1389,  0.4363],
        [ 3.1232, -2.4830],
        [-3.1352,  2.9283],
        [-3.1289, -0

In [ ]:

file_model = file_path.replace(".cif","")
pdbl = PDBList()
cif_parser = MMCIFParser()
structure = cif_parser.get_structure("protein",'/content/target directory/1axj.cif')
model0 = structure[0]
chain_A = model0['A']

structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('pred.pdb',  preserve_atom_numbering=True)

Ignore from here on

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output
class CustomLoss(nn.Module):
    def __init__(self, predicted_angles, angles_tensor):
        super(CustomLoss, self).__init__()
        self.predicted_angles= predicted_angles
        self.angles_tensor= angles_tensor
    def forward(self):
        print(self.predicted_angles)
        print(self.angles_tensor)
        d_list = []
        for i in range(len(self.angles_tensor)):
            x1, y1 = self.predicted_angles[0][i]
            x2, y2 = self.angles_tensor[i]
            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))
            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))
            d = torch.sqrt(ax_x**2 + ax_y**2)
            d_list.append(d)
        d_tensor = torch.stack(d_list)
        return d_tensor.mean()
class TransformerTrainer:
    def __init__(self, model: nn.Module, CustomLoss: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.CustomLoss = CustomLoss
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.SGD(model.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            print(self.angles_tensor)
            loss = CustomLoss()
            loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            #loss_list.append(loss.item())
            print(loss)
            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss


In [ ]:
#with  using mean angle error loss directly
class TransformerTrainer2:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            loss.backward(retain_graph=True)
            #for name, param in self.model.named_parameters():
            #    if param.grad is not None:
            #        print(f"Gradient for {name}: {param.grad.norm()}")

            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss, self).__init__()

    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] - angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)
        #print(predictions)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)


        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return mse

In [ ]:
#with  using mean angle error loss with mse loss
class TransformerTrainer3:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T).float()
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)
        self.empty_tensor = torch.zeros_like(self.angles_tensor)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            mean_angle_error = MeanAngleErrorLoss2()(predictions.squeeze(), self.angles_tensor)
            loss = self.criterion(mean_angle_error, self.empty_tensor)


            loss.backward(retain_graph=True)
            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss2(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss2, self).__init__()


    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] -angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)

        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        #mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return error_combined

In [ ]:
#checking whats wrong with  angle loss, by checking gradients and scaling to mse loss
class TransformerTrainer4:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            #loss.backward(retain_graph=True)
            custom_loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            scaled_custom_loss = 2 * custom_loss

            # Compute MSE loss for comparison
            mse_loss = nn.MSELoss()(predictions.squeeze(), self.angles_tensor)
            scaled_custom_loss = scaled_custom_loss + mse_loss

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Scaled Custom Loss: {scaled_custom_loss.item()}, MSE Loss: {mse_loss.item()}")

            scaled_custom_loss.backward(retain_graph=True)


            #for name, param in self.model.named_parameters():
            #    if param.grad is not None:
            #        print(f"Gradient for {name}: {param.grad.norm()}")

            self.optimizer.step()
            loss_list.append(scaled_custom_loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {scaled_custom_loss.item()}")

In [ ]:
class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor):
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

        loss = loss_phi+ loss_psi
        print(loss)
        return loss

class TransformerTrainer5:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss



In [ ]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 20
model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
#model.forward(embeddings)
criterion = MeanAngleErrorLoss()
num_epochs = 20
#criterion = nn.MSELoss()
#criterion = nn.L1Loss()

#criterion  = AngularLoss()
trainer = TransformerTrainer4(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model